<a href="https://colab.research.google.com/github/DaniellePotts/Sentiment-Analysis-with-QReLU-and-mQReLU/blob/main/sentiments_140_ReLU_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install contractions
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 2.8 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
%tensorflow_version 1.x 
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# from QReLU_TensorFlow_Keras import QReLU
# from m_QReLU_TensorFlow_Keras import m_QReLU

import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import spacy
import re
import unicodedata
import nltk
nltk.download('stopwords')
import contractions
from collections import Counter
from nltk.tokenize.toktok import ToktokTokenizer

from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [ ]:
nlp = spacy.load('en', parse=True, tag=True, entity=True)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

In [ ]:
sentiments_cat = ['negative','neutral','postive']

In [ ]:
def chop_dataframe(df, amount):
  df_4 = df.loc[df['sentiment'] == 4]
  df_1 = df.loc[df['sentiment'] == 0]
  df_final = df_4.head(amount)
  return pd.concat([df_final, df_1.head(amount)])

def encode_corpus(corpus):
  words = ' '.join(corpus).split()
  
  count_words = Counter(words)
  total_words = len(words)

def count_words(corpus):
  words = ' '.join(corpus).split()
  
  counted_words = Counter(words)
  total_words = len(words)

  return total_words, counted_words

def sort_words(corpus):
  total_words, counted_words = count_words(corpus)
  return counted_words.most_common(total_words)

def decode_sentiments(encoded_sentiments, sentiments_cat=sentiments_cat, postive_index=2, 
                      negative_index=0, neutral_index=1):
  decoded_sentiments = []

  for index in range(0, len(encoded_sentiments)):
    if encoded_sentiments[index] == 0:
      decoded_sentiments.append(sentiments_cat[negative_index])
    elif encoded_sentiments[index] == 2:
      decoded_sentiments.append(sentiments_cat[neutral_index])
    elif encoded_sentiments[index] == 4 or encoded_sentiments[index] == 1:
      decoded_sentiments.append(sentiments_cat[postive_index])
  
  return decoded_sentiments

def convert_sentiments(sentiments):
  converted = []

  for s in sentiments:
    if s == 4:
      converted.append(1)
    else:
      converted.append(0)
  return np.array(converted)

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

def convert_to_lower(corpus):
    return [c.lower() for c in corpus]
    
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def expand_contractions(text):
  expanded_words = []    
  for word in text.split():
    # using contractions.fix to expand the shotened words
    expanded_words.append(contractions.fix(word))   
      
  return' '.join(expanded_words)

In [ ]:
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        
        # lemmatize text
        # if text_lemmatization:
        #     doc = lemmatize_text(doc)
        
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
            
            pat1 = r'@[A-Za-z0-9_]+'
            pat2 = r'https?://[^ ]+'
            www_pat = r'www.[^ ]+'
            combined_pat = r'|'.join((pat1, pat2))
            doc = re.sub(combined_pat, '', doc)
            doc = re.sub(www_pat, '', doc)

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [ ]:
cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv("./drive/MyDrive/QReLU/training.1600000.processed.noemoticon.csv", header=None, names =cols, engine='python')

In [ ]:
corpus = convert_to_lower(df['text'].values)
normalized = normalize_corpus(corpus)

print(df[df['sentiment'] == 0].size)
print(df[df['sentiment'] == 4].size)

4800000
4800000


In [ ]:
df['normalized'] = normalized

In [ ]:
df.head()

,sentiment,id,date,query_string,user,text,normalized
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot httptwitpic comyzl bummer shoulda g...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset cannot update facebook texting might cry...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dived many times ball managed save re...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass no not behaving mad cannot see


In [ ]:
max_words = 663204

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=max_words, split=' ')
tokenizer.fit_on_texts(df['text'].values)

In [ ]:
def convert_sentiments(sentiments):
  converted = []

  for s in sentiments:
    if s == 4:
      converted.append(1)
    else:
      converted.append(0)
  return np.array(converted)

In [ ]:
df['encoded_sentiments'] = convert_sentiments(df['sentiment'].values)

In [ ]:
max_length = 160

In [ ]:
X = tokenizer.texts_to_sequences(df['normalized'])

In [ ]:
X = sequence.pad_sequences(X, max_length, padding='post')

In [ ]:
len(X)

1600000

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, df['encoded_sentiments'].values, test_size=0.2, random_state=42)

In [ ]:
len(X_train), len(X_test), len(y_train), len(y_test)

(1280000, 320000, 1280000, 320000)

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_words, 100, input_length=max_length),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='relu')
])

model2.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 160, 100)          66320400  
_________________________________________________________________
conv1d (Conv1D)              (None, 160, 32)           9632      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (N

In [28]:
model2.compile(loss="binary_crossentropy",optimizer="adam",metrics=['acc'])

history = model2.fit(X_train, y_train, epochs=5, validation_split=0.2, batch_size=32, shuffle=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1024000 samples, validate on 256000 samples
Epoch 1/5
1024000/1024000 [==============================] - 5878s 6ms/sample - loss: 7.7051 - acc: 0.5005 - val_loss: 7.7116 - val_acc: 0.5001
Epoch 2/5
1024000/1024000 [==============================] - 5911s 6ms/sample - loss: 7.7051 - acc: 0.5005 - val_loss: 7.7116 - val_acc: 0.5001
Epoch 3/5
1024000/1024000 [==============================] - 5839s 6ms/sample - loss: 7.7051 - acc: 0.5005 - val_loss: 7.7116 - val_acc: 0.5001
Epoch 4/5
1024000/1024000 [==============================] - 5867s 6ms/sample - loss: 7.7051 - acc: 0.5005 - val_loss: 7.7116 - val_acc: 0.5001
Epoch 5/5
1024000/1024000 [==============================] - 5895s 6ms/sample - loss: 7.7051 - acc: 0.5005 - val_loss: 7.7116 - val_acc: 0.5001


In [29]:
results = model2.evaluate(X_test, y_test)
print(results)

320000/320000 [==============================] - 351s 1ms/sample - loss: 7.7369 - acc: 0.4984
[7.736864941120148, 0.49841875]
